In [91]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import matplotlib

In [128]:
matplotlib.rcParams.update({'font.size': 12})
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

In [287]:
 #### Read Train and Test Data
training_data = pd.read_csv('/home/rajneesh/Desktop/Queen_City_Hackathon/training.csv', index_col=0)
testing_data = pd.read_csv('/home/rajneesh/Desktop/Queen_City_Hackathon/testing.csv', index_col=0)

'''
### Check for the co relared columns 

# Create correlation matrix
# Create correlation matrix
corr_matrix = training_data.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
columns_to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]


### Drop all the co-related columns
data_nodups_df = training_data.drop(columns=columns_to_drop)
'''
data_nonull_df = training_data.fillna(training_data.mean())
y = data_nonull_df['target']
X = data_nonull_df.drop(columns=['target'])

In [288]:
from sklearn.decomposition import PCA
pca = PCA(n_components=300)
pca_data = pca.fit_transform(X)
#pca.explained_variance_ratio_.cumsum()

In [289]:
st_scale = StandardScaler()
X_train, X_test, y_train, y_test=train_test_split(pca_data, y, test_size=0.30)

In [290]:
parameters_for_testing = {
    'colsample_bytree':[0.4,0.6,0.8],
    'gamma':[0, 0.03, 0.1, 0.3],
    'min_child_weight':[1.5, 4, 6, 8, 10],
    'learning_rate':[0.1, 0.3, 0.07, 0.01],
    'max_depth':[3, 5, 7],
    'n_estimators':[50, 100, 200, 300, 500],
    'reg_alpha':[1e-5, 1e-2,  0.75],
    'reg_lambda':[1e-5, 1e-2, 0.45],
    'subsample':[0.6,0.95]  
}

xgb_model = xgb.XGBRegressor(learning_rate =0.1, n_estimators=100, max_depth=5, min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, nthread=6, scale_pos_weight=1, seed=27)
gsearch1 = RandomizedSearchCV(estimator = xgb_model, param_distributions= parameters_for_testing, n_jobs=-1, iid=False, verbose=10, scoring='neg_mean_squared_error')
gsearch1.fit(X_train, y_train)
print('best params')
print (gsearch1.best_params_)
print('best score')
print (gsearch1.best_score_)

/home/rajneesh/miniconda3/envs/py36/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  19 out of  30 | elapsed:    9.3s remaining:    5.4s
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:   10.6s remaining:    3.2s
[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed:   11.5s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   12.3s finished
/home/rajneesh/miniconda3/envs/py36/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[02:00:59] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'subsample': 0.95, 'reg_lambda': 0.01, 'reg_alpha': 0.01, 'n_estimators': 50, 'min_child_weight': 1.5, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 0.1, 'colsample_bytree': 0.8}
best score
-0.7469941434125701


In [292]:
#best_params = {"objective":"reg:squarederror", 'subsample': 0.95, 'reg_lambda': 1e-05, 'reg_alpha': 1e-05, 'n_estimators': 300, 'min_child_weight': 1.5, 'max_depth': 7, 'learning_rate': 0.07, 'gamma': 0.03, 'colsample_bytree': 0.4}
# best_params = {'subsample': 0.95, 'reg_lambda': 1e-05, 'reg_alpha': 0.01, 'n_estimators': 100, 'min_child_weight': 10, 'max_depth': 5, 'learning_rate': 0.1, 'gamma': 0.3, 'colsample_bytree': 0.4}
# best_params = {'subsample': 0.95, 'reg_lambda': 1e-05, 'reg_alpha': 1e-05, 'n_estimators': 200, 'min_child_weight': 1.5, 'max_depth': 7, 'learning_rate': 0.1, 'gamma': 0.3, 'colsample_bytree': 0.8}
best_params = {'subsample': 0.95, 'reg_lambda': 0.01, 'reg_alpha': 0.01, 'n_estimators': 50, 'min_child_weight': 1.5, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 0.1, 'colsample_bytree': 0.8}

params = {"objective":"reg:squarederror",'colsample_bytree': 0.5,'learning_rate': 0.05, 'max_depth': 5, 'alpha': 10, 'n_estimators': 200}
train_dmatrix = xgb.DMatrix(data=X_train, label=y_train)
xg_reg = xgb.train(params=params, dtrain=train_dmatrix, num_boost_round=1000)

In [297]:
y_pred = xg_reg.predict(xgb.DMatrix(data=X_test))

In [299]:
mse = mean_squared_error(y_test, y_pred)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % np.sqrt(mse))

MSE: 0.06
RMSE: 0.25


In [300]:
mse = mean_squared_error(y_test, y_pred)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % np.sqrt(mse))

MSE: 0.06
RMSE: 0.25


In [159]:
xg_reg = xgb.train(params=params, dtrain=xgb.DMatrix(data=pca_data, label=y), num_boost_round=1000)

/home/rajneesh/miniconda3/envs/py36/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/rajneesh/miniconda3/envs/py36/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [301]:
#test_nodups_df = testing_data.drop(columns=columns_to_drop)
test_nodups_df = testing_data.fillna(training_data.mean())

pca_test = pca.transform(test_nodups_df)

In [302]:
y_pred = xg_reg.predict(xgb.DMatrix(data=pca_test))

In [184]:
def save_prediction_to_csv(y_pred):
    """
    Use this function to save your prediction result to a csv file.
    The resulting csv file is named as [team_name].csv

    :param y_pred: an array or a pandas series that follows the SAME index order as in the testing data
    """
    pd.DataFrame(dict(
        target=y_pred
    )).to_csv('predictions.csv', index=False, header=False)

In [303]:
save_prediction_to_csv(y_pred)

In [304]:
y_pred.mean()

0.21879552